In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
os.chdir("..")

In [2]:
LOGS_ROOT_DIR = Path("lightning_logs")

In [3]:
logs_raw_paths = list(LOGS_ROOT_DIR.rglob("**/*.csv"))
logs_raw_paths

[PosixPath('lightning_logs/MNISTClassificationModule_MNISTProvider/raw_results.csv'),
 PosixPath('lightning_logs/MNISTIdentityModule_MNISTProvider/raw_results.csv'),
 PosixPath('lightning_logs/MNISTMappingModule_MNISTProvider/raw_results.csv')]

In [4]:
logs_path = logs_raw_paths[0]
logs_path.parts[1]

'MNISTClassificationModule_MNISTProvider'

In [5]:
df = pd.read_csv(logs_path)

In [6]:
df.head()

,accuracy,val_loss,experiment,model,experiment_name,hidden_size,num_hidden,seed
0,0.970186,0.079579,MNISTClassificationModule_MNISTProvider,DepthwiseAutomorpherNet,20240812_h128_one_hidden_s79,128,one_hidden,79
1,0.919628,0.276078,MNISTClassificationModule_MNISTProvider,DepthwiseAutomorpherNet,20240812_h128_identity_s19,128,identity,19
2,0.919152,0.276468,MNISTClassificationModule_MNISTProvider,DepthwiseAutomorpherNet,20240812_h128_identity_s17,128,identity,17
3,0.970218,0.077968,MNISTClassificationModule_MNISTProvider,DepthwiseAutomorpherNet,20240812_h128_one_hidden_s53,128,one_hidden,53
4,0.970206,0.082511,MNISTClassificationModule_MNISTProvider,DepthwiseAutomorpherNet,20240812_h128_one_hidden_s37,128,one_hidden,37


In [7]:
for col in ["accuracy", "val_loss"]:
    df[f"{col}_mean"] = df[col].copy()
    df = df.rename(columns={col:f"{col}_stddev"})

In [12]:
aggregated = df.groupby(["hidden_size", "num_hidden", "model"])\
    .agg(
        {
            "accuracy_mean": np.mean, 
            "accuracy_stddev": np.std, 
            "val_loss_mean": np.mean, 
            "val_loss_stddev": np.std
        })

/var/folders/j_/gy0508_n2w36wpr12gx3tz2m0000gn/T/ipykernel_62445/46863417.py:2: FutureWarning: The provided callable <function mean at 0x105d10940> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .agg(
/var/folders/j_/gy0508_n2w36wpr12gx3tz2m0000gn/T/ipykernel_62445/46863417.py:2: FutureWarning: The provided callable <function std at 0x105d10a60> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg(
/var/folders/j_/gy0508_n2w36wpr12gx3tz2m0000gn/T/ipykernel_62445/46863417.py:2: FutureWarning: The provided callable <function mean at 0x105d10940> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .agg(


In [13]:
aggregated

accuracy_mean  \
hidden_size num_hidden   model                                    
128         identity     AutomorpherNet                0.920673   
                         BaselineNet                   0.975770   
                         DepthwiseAutomorpherNet       0.919694   
            one_hidden   AutomorpherNet                0.968527   
                         BaselineNet                   0.976621   
                         DepthwiseAutomorpherNet       0.970506   
            three_hidden AutomorpherNet                0.970692   
                         BaselineNet                   0.975890   
                         DepthwiseAutomorpherNet       0.971767   

                                                  accuracy_stddev  \
hidden_size num_hidden   model                                      
128         identity     AutomorpherNet                  0.000334   
                         BaselineNet                     0.000451   
                         DepthwiseAutomorpherNet         0.000389   
            one_hidden   AutomorpherNet                  0.000443   
                         BaselineNet                     0.001404   
                         DepthwiseAutomorpherNet         0.000597   
            three_hidden AutomorpherNet                  0.000515   
                         BaselineNet                     0.000172   
                         DepthwiseAutomorpherNet         0.000424   

                                                  val_loss_mean  \
hidden_size num_hidden   model                                    
128         identity     AutomorpherNet                0.273682   
                         BaselineNet                   0.071844   
                         DepthwiseAutomorpherNet       0.276123   
            one_hidden   AutomorpherNet                0.087616   
                         BaselineNet                   0.075363   
                         DepthwiseAutomorpherNet       0.080196   
            three_hidden AutomorpherNet                0.086051   
                         BaselineNet                   0.081150   
                         DepthwiseAutomorpherNet       0.083495   

                                                  val_loss_stddev  
hidden_size num_hidden   model                                     
128         identity     AutomorpherNet                  0.001252  
                         BaselineNet                     0.000565  
                         DepthwiseAutomorpherNet         0.000602  
            one_hidden   AutomorpherNet                  0.003128  
                         BaselineNet                     0.002265  
                         DepthwiseAutomorpherNet         0.003184  
            three_hidden AutomorpherNet                  0.003183  
                         BaselineNet                     0.002926  
                         DepthwiseAutomorpherNet         0.001287

In [14]:
print(aggregated.to_latex())

\begin{tabular}{lllrrrr}
\toprule
 &  &  & accuracy_mean & accuracy_stddev & val_loss_mean & val_loss_stddev \\
hidden_size & num_hidden & model &  &  &  &  \\
\midrule
\multirow[t]{9}{*}{128} & \multirow[t]{3}{*}{identity} & AutomorpherNet & 0.920673 & 0.000334 & 0.273682 & 0.001252 \\
 &  & BaselineNet & 0.975770 & 0.000451 & 0.071844 & 0.000565 \\
 &  & DepthwiseAutomorpherNet & 0.919694 & 0.000389 & 0.276123 & 0.000602 \\
\cline{2-7}
 & \multirow[t]{3}{*}{one_hidden} & AutomorpherNet & 0.968527 & 0.000443 & 0.087616 & 0.003128 \\
 &  & BaselineNet & 0.976621 & 0.001404 & 0.075363 & 0.002265 \\
 &  & DepthwiseAutomorpherNet & 0.970506 & 0.000597 & 0.080196 & 0.003184 \\
\cline{2-7}
 & \multirow[t]{3}{*}{three_hidden} & AutomorpherNet & 0.970692 & 0.000515 & 0.086051 & 0.003183 \\
 &  & BaselineNet & 0.975890 & 0.000172 & 0.081150 & 0.002926 \\
 &  & DepthwiseAutomorpherNet & 0.971767 & 0.000424 & 0.083495 & 0.001287 \\
\cline{1-7} \cline{2-7}
\bottomrule
\end{tabular}

